In [1]:
# notebook devoted for heavy GPU jobs
from huggingface_hub import login
import transformers
import numpy as np
import nltk
from nltk.corpus import gutenberg
import torch
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
login(token="YOUR API TOKEN")

In [2]:
# article
prompt_with_the_train = "The cat sat on the mat by the door, watching the birds fly over the trees in the garden while the wind blew through the open window. The man walked into the store, picked up the newspaper from the rack, and glanced at the headline about the mayor’s speech in the city square, just as the clock struck noon."
prompt_with_the_test = "The sound of the wind whistling through the trees echoed across the valley as the sun began to dip below the horizon, casting long shadows on the ground, while the birds, startled by the sudden rustling of the leaves, flew toward the safety of the dense forest, where the last rays of light barely touched the tops of the tallest trees, and the distant howl of the wolves signaled the beginning of the night."

# nouns
prompt_with_cat_train = "As I walked into the room, I was greeted by a sea of cats—black cats, white cats, fluffy cats, skinny cats, and cats of every size imaginable. There were cats lounging on the couch, cats perched on the windowsill, and even a few adventurous cats climbing the bookshelves. In the corner, a group of curious kittens played with a ball of yarn, while older cats watched lazily, their tails swishing in contentment. It seemed like every inch of the space was filled with the soft purrs and playful meows of the countless cats around me."
prompt_with_cat_test = "The garden was overrun with cats—cats darting between the flower beds, cats sunbathing on the stone path, and cats curling up in the shade of the large oak tree. A trio of sleek tabby cats chased each other around the fountain, while a pair of fluffy Persian cats lazily pawed at a butterfly. In the distance, a group of ginger cats sat in a perfect line, watching the scene with regal indifference. Everywhere I turned, there were cats—perched on walls, nestled in baskets, or simply lounging in the tall grass, their eyes half-closed in sleepy satisfaction."

# prepositions
prompt_with_prep_train = "On a quiet morning, I walked through the garden and past the old oak tree, stopping beside the stone bench near the pond. Birds fluttered above the trees, while a squirrel darted between the bushes, disappearing into a hole in the ground. I placed my bag under the bench, then sat down with a book in hand, feeling the breeze against my face. As the sun rose higher in the sky, casting light over the grass, I glanced toward the horizon, wondering what lay beyond the hills."
prompt_with_prep_test = "At dawn, I wandered along the riverbank, through the tall grass, and beneath the branches of the willow trees. The mist hung low over the water, drifting around the rocks scattered near the shore. I walked toward the bridge, leaning on the railing as I looked across the flowing stream. Above me, birds swooped in circles, and beyond the hills, the sky slowly turned pink. I stayed there until the first rays of sunlight fell on the water, casting reflections beneath the trees."

# prompt with the top 100 most frequent word in english
prompt_frequent_words_train = "In today’s world, people often feel the push and pull of connection and solitude. With technology and social media on the rise, we now have countless ways to stay in touch with those we know and love. However, the question of whether this digital world can truly satisfy our need for real connection remains. To be truly connected is to share experiences, to understand and to be understood. This kind of connection goes far beyond a screen. The desire to connect is universal, and people have searched for it throughout history. In ancient times, communities formed to support one another, to live together, and to build bonds that could help them through challenges. Today, people may still crave this closeness, yet it is not always easy to find in our fast-paced world. While the internet gives us access to almost anyone, anywhere, it does not always give us the depth of connection that true friendship and family relationships can offer. Consider a family spending time together. For many people, family is a source of strength, a place where they feel safe and understood. But as life becomes busier, it can be easy to let work, hobbies, or other commitments pull us away from family time. Many people find that they must make a conscious effort to set aside time for their loved ones. To sit down for a meal together, to talk about the day, to share thoughts and laughter — these moments are priceless. They are what remind us of who we are and who we want to become. Friendship, too, plays a significant role in life. Friends are often the people we choose to spend time with, the ones who share our interests and support us. To have friends is to feel understood in a unique way, to laugh together, to encourage each other, and sometimes just to sit in silence knowing someone is there. A true friend listens without judgment, stands by us in hard times, and celebrates with us in good times. However, maintaining friendships can require work and commitment. With busy lives, people can sometimes lose touch, only to realize later how much those friendships meant. In the workplace, relationships are equally important. Many people spend a significant amount of time at work, so having good connections there can make a big difference. Working with others requires collaboration, understanding, and respect. When people feel connected to their colleagues, they tend to work better together, share ideas freely, and support each other. A positive work environment can foster not only productivity but also well-being. People are more likely to feel satisfied in their work when they know they are valued and understood by those around them. Of course, technology plays a large role in modern connections. Platforms like social media allow people to connect across distances, to share life events, and to communicate instantly. For some, these tools make it easier to stay in touch with family and friends, to share news, and to express themselves. However, while technology can bring people closer, it can also create a sense of distance. Seeing the lives of others through a screen is different from experiencing life together in person. The highlights of life shared online may not always show the full picture, leaving people to wonder if they are missing out. To have genuine connection, people often need to go beyond what is easy and convenient. Sometimes, this means reaching out, making an effort, and being open. True connection requires understanding and empathy. It asks us to listen, to be present, and to care. In a world that often moves fast, taking the time to connect deeply can feel like a challenge, but it is also incredibly rewarding. The value of connection is evident in difficult times. When people go through challenges, it is often those close connections that help them through. Whether it’s a friend who listens, a family member who offers support, or a colleague who steps in to lend a hand, these connections give people strength. Knowing that someone else understands or is there to help can make all the difference. Moreover, the ability to connect also fosters compassion. When people share experiences, they begin to see the world from each other’s perspectives. This understanding can lead to greater kindness and less judgment. People who feel connected are often more empathetic, more understanding, and more willing to help others. This creates a positive cycle, as kindness and empathy tend to inspire more of the same. For people to have a balanced and fulfilled life, connection is essential. But to nurture these connections takes effort. It is not always easy to set aside time, to reach out, or to stay in touch. Life can be busy, and distractions are everywhere. However, those who make the effort to connect often find that their lives are richer and more satisfying. The joy of shared laughter, the comfort of understanding, and the strength of support are all things that make life meaningful. As we move through life, the connections we make help to shape who we are. We learn from others, grow with them, and find new perspectives. Each person we meet adds to our experience and helps us to see the world in new ways. Sometimes, people find that their most valuable lessons come from those who are different from them. To connect with people from various backgrounds and with different life experiences is to broaden our view of the world. In conclusion, to live fully is to connect meaningfully. Whether through family, friends, colleagues, or even strangers, these bonds enrich life. They offer joy, comfort, and understanding, and they remind us that we are not alone. While technology may change the way we communicate, it cannot replace the depth of real connection. To make time for those who matter, to share moments, and to care is to live a life of purpose and love."
prompt_frequent_words_test = "In the world we live in, each day is filled with choices we all make, big or small. The way we approach these choices can be what shapes not only our own lives but also the lives of those around us. To be the kind of person who reflects on what they do, who they are, and who they want to become, is to take a meaningful step toward self-awareness and growth. One of the first things to know about making decisions is that they are all interconnected. When we choose to do one thing, it often means we cannot do another. This may seem obvious, but to understand the full impact of this reality, it helps to look at the ways in which choices play out in real life. We are always presented with options, and while some decisions may seem trivial at first, they add up over time. We can think about it like this: to choose a path, even if small, can set in motion a series of events that shape our lives in ways we could never fully predict. For example, people make decisions on how to spend their time. Time is one of the most valuable resources we have. There is no getting more of it, and once it’s gone, it’s gone for good. How we choose to spend it — whether working, relaxing, being with family, or helping others — says a lot about what we value. Some people may spend time worrying about things that, in the end, are not as important as they seem, while others may put time into making themselves or others better. This shows the difference in what people find to be meaningful or valuable in life. When we look at those around us, we see that everyone is trying to figure out what it means to live a good life. Some believe that to have a successful career is key, while others might think that family or friendships are the foundation of a fulfilling life. Whatever the focus may be, it’s clear that we all want a life filled with purpose. The concept of purpose is one that everyone seems to look for, although it can mean different things to different people. People sometimes find that purpose comes from the roles they play in the lives of others. For instance, many parents feel that to have children and raise them well is one of the most meaningful things they can do. They look to guide their children, to give them the tools they need to make their own choices. The idea of helping others extends beyond family, as people also contribute to their communities in many ways. Volunteering, supporting friends, and giving back are just some ways people find meaning in their lives. In work, too, people seek purpose. It’s common to find that people want to do something that they can be proud of, something that allows them to use their talents and contribute to society. This desire to work well is what drives many people forward and keeps them motivated. Yet, work can also become overwhelming, especially when people forget to balance their time between work and other areas of life. Balance is essential in any good life. We often have to remind ourselves not to let any one part of life take up all of our attention. It’s not easy, but it is essential if we want to live fully. This balance extends to how people think about success. For some, success is about achieving certain goals, like owning a home, getting a promotion, or earning a certain amount of money. For others, success is about having good relationships, feeling at peace, and being happy. Each person will have their own idea of what it means to succeed. Some may be quick to compare themselves to others, thinking that if someone else has something they don’t, they are somehow lacking. But to make comparisons is not often helpful. We each have our own journey, and to look too much at what others have can take away from the joy of our own experiences. Another important part of life is facing challenges. There are times when things don’t go as planned, and it’s easy to feel frustrated. However, these moments are often when we learn the most about ourselves. Challenges can show us what we are capable of and remind us that we are stronger than we think. People often say that to know hardship is to know strength, and it’s true that the challenges we face can make us wiser and more resilient. People also face choices about how they treat those around them. Kindness, empathy, and patience are qualities that many strive to have, but it’s not always easy to be kind in a busy, fast-paced world. However, those who make a habit of treating others with respect and understanding often find that they are happier and more fulfilled. Relationships, whether with family, friends, or colleagues, require effort and care. By giving time to nurture these connections, we make life richer not only for ourselves but also for others. In moments of reflection, people often think about their lives and ask themselves, “Am I doing what I want to do? Am I living the life I want to live?” These questions can be difficult to answer, but they are important. To pause and look within is to take stock of what matters. This reflection can help guide future decisions and keep people on the path that is right for them. At the end of the day, life is made up of moments, decisions, and interactions. To be mindful of how we choose to spend these moments, of who we choose to spend them with, and of what we put into the world, is to live with purpose. While each person’s path may be unique, we all share the desire to find happiness and meaning. In seeking to make each day count, to treat others well, and to pursue goals that matter to us, we contribute to a world that is better for all. It may not always be easy, and we may not always make the right choices. But to try, to reflect, and to grow from each experience, is to live a life well-lived. We each have the power to make choices that, in time, build a story of who we are and what we stand for. Let that story be one of kindness, purpose, and joy, shared with those who mean the most to us."

# words of similar meanings, their embedding space should be similar too 
prompt_with_cheesecake_train = "Cheese is one of the most versatile ingredients in the culinary world, and cheese can be used in everything from savory dishes to desserts. Cheese lovers often enjoy pairing cheese with crackers, wine, or fruit, but cheese also shines in baking. Cake, on the other hand, is the quintessential dessert, with cake being a staple at celebrations. Cake comes in many forms, such as chocolate cake, vanilla cake, or even carrot cake. However, when you bring cheese and cake together to create cheesecake, a magical transformation happens. Cheesecake is a dessert like no other, with cheesecake offering the creaminess of cheese and the sweetness of cake in perfect harmony. Cheesecake can be topped with fruits like strawberries or blueberries, or cheesecake can be flavored with chocolate or caramel. Some people prefer classic cheesecake, while others enjoy a more decadent cheesecake loaded with toppings. Regardless of the variation, cheesecake remains one of the most beloved desserts worldwide. The crust of cheesecake, often made from crushed biscuits or graham crackers, complements the smooth filling, making cheesecake irresistible. Cheese plays a central role in cheesecake, while the influence of cake ensures that cheesecake is always a delightful dessert. Whether you love cheese, crave cake, or are obsessed with cheesecake, this dessert proves that the combination of cheese and cake is truly extraordinary. Every bite of cheesecake reminds us that cheese and cake, when united in cheesecake, are a match made in heaven. Cheesecake aficionados often debate whether baked cheesecake or no-bake cheesecake is superior, but all agree that cheesecake is a dessert worth savoring. With so many variations, cheesecake enthusiasts never tire of exploring new ways to enjoy their favorite dessert. Cheese and cake come together seamlessly in cheesecake, showing how cheese and cake can create something greater than their individual parts. Cheesecake is, without a doubt, the ultimate testament to the greatness of cheese and cake in unison."
prompt_with_cheesecake_test = "Cheese is a culinary treasure that has delighted taste buds for centuries. Whether it’s creamy, tangy, or sharp, cheese offers endless possibilities. Cheese finds its way into countless dishes, from savory casseroles to gooey pizzas, and its versatility knows no bounds. Cake, too, is a universal favorite, with cake symbolizing joy, celebration, and indulgence. Cake comes in every flavor imaginable—chocolate cake, vanilla cake, red velvet cake—and each cake brings its own special charm. But when cheese and cake are combined to form cheesecake, something truly extraordinary happens. Cheesecake is a dessert that transcends expectations, merging the velvety richness of cheese with the sweet, airy allure of cake. Cheesecake can be baked or chilled, simple or elaborate, yet every cheesecake captures the perfect balance of flavors. Classic cheesecake recipes highlight the creamy taste of cheese, while fruit-topped cheesecake adds a burst of freshness. Some people swear by chocolate cheesecake or caramel-drizzled cheesecake, while others can’t resist a zesty lemon cheesecake. No matter the variation, cheesecake consistently proves that the union of cheese and cake is a match made in heaven. The crust of cheesecake, typically crafted from crushed cookies or graham crackers, provides the ideal foundation for the smooth and luscious cheese layer. Every bite of cheesecake reminds us why this dessert has stood the test of time. Fans of cheese, cake, and cheesecake alike agree that cheesecake combines the best of both worlds. Whether you’re indulging in a slice of classic cheesecake, exploring new cheesecake flavors, or savoring the rich taste of a perfectly baked cheesecake, it’s clear that cheese and cake reach their pinnacle when united in cheesecake. Cheesecake is a testament to how cheese and cake, when brought together, create something greater than the sum of their parts. From the first bite to the last, cheesecake is a celebration of everything wonderful about cheese, cake, and, of course, cheesecake itself."

# same word with different meaning at different contexts 
prompt_with_short_sentences_train = "The sail was patched with flour sacks. It looked like the flag of permanent defeat. But the old man was not defeated. He carried the mast on his shoulder and walked away. The boy watched him go. He loved the old man. He was rowing steadily and it was no effort for him. The sun was hot now although the breeze was rising gently. 'I feel confident today,' the old man said. 'I feel confident.' The small fish were swimming near the boat. He watched them as he rowed. They moved quickly and gracefully."
prompt_with_short_sentences_test = "The cat slept. He opened the door. The sky is blue. She smiled. The sun set."

# llama3 versus 3 llamas:
prompt_with_llama3llamas_train = "Once upon a time, there was a groundbreaking artificial intelligence system known as Llama3. It was designed to tackle the world's most intricate challenges, including understanding the behavior of animals. One day, Llama3 received an unusual but exciting mission: to study the interactions of 3 llamas living on a lush mountainside. The 3 llamas, named Luna, Leo, and Lila, were an inseparable trio. Llama3 was tasked with observing their daily lives to uncover patterns in their social behavior. Early each morning, Luna would lead the 3 llamas to a sunlit meadow. Using its advanced analytical capabilities, Llama3 noticed that the 3 llamas had a peculiar system of taking turns guarding the group while the others grazed. Llama3 identified that Luna, the leader, always took the first watch, followed by the playful Leo, and finally the cautious Lila. As the days passed, Llama3 learned more about the 3 llamas’ communication. Although they couldn’t speak, their subtle ear twitches, hums, and movements told intricate stories. Llama3 mapped out how the 3 llamas shared information, such as warning each other about predators or signaling when it was time to head back to their sheltered valley. By processing hours of data, Llama3 discovered that the 3 llamas had a unique friendship dynamic: Luna was the brave leader, Leo the jokester who lightened tense moments, and Lila the gentle peacemaker who resolved any disagreements among the 3 llamas. In the end, Llama3’s findings amazed researchers. It turned out that the seemingly simple interactions of the 3 llamas held profound lessons about cooperation, trust, and community. Llama3 not only unveiled the hidden lives of Luna, Leo, and Lila but also helped humans better understand how to foster harmony in their own lives. And so, thanks to Llama3 and the 3 llamas, the world gained a deeper appreciation for the beauty of teamwork and connection."
prompt_with_llama3llamas_test = "Deep in the Andean highlands, a team of scientists deployed their latest invention, Llama3, a sophisticated AI designed to decode animal behavior. They placed Llama3 in the middle of a vibrant alpaca farm, but its primary focus was not on the dozens of alpacas. Instead, it was assigned to study 3 llamas who roamed the farm’s outskirts—Pacha, Sol, and Tika. These 3 llamas were rumored to have an uncanny ability to navigate the rugged terrain and a mysterious bond that seemed almost telepathic. The scientists were fascinated by these 3 llamas and hoped Llama3 could unravel the secrets of their dynamic. Armed with cameras, sensors, and its advanced algorithms, Llama3 began observing. Pacha, the largest of the 3 llamas, was the guardian. He always stayed on high ground, scanning for predators. Sol, known for her curiosity, frequently wandered off, often finding the richest patches of grass or hidden streams. Tika, the smallest and youngest, was the glue that held them together. Whenever Sol wandered too far, Tika would nudge her back toward the group. Llama3 not only recorded these patterns but also began predicting their actions. It noticed that the 3 llamas communicated with subtle ear flicks and humming sounds, which Llama3 translated into a rudimentary code of warnings, directions, and reassurances. Llama3 even predicted that Pacha would lead the 3 llamas to a new grazing spot just before a sudden rainstorm—a decision the scientists later marveled at, as it showed a surprising awareness of weather changes. But the real breakthrough came one night when the 3 llamas saved the farm’s herd from a predator. Using its night-vision capabilities, Llama3 recorded how Pacha stood guard, Sol herded the younger animals into safety, and Tika sprinted to alert the humans. The 3 llamas worked seamlessly, their roles perfectly aligned, as if they had rehearsed this scenario a hundred times. The next morning, the scientists reviewed Llama3’s data and were astonished by the depth of cooperation and intelligence displayed by the 3 llamas. They realized that their invention, Llama3, was not just observing animals but learning from them. The insights from Llama3 and the 3 llamas inspired the team to apply these lessons to human technology, creating systems that could adapt and collaborate as effortlessly as the trio on the farm. And so, the story of Llama3 and the 3 llamas became a legend of innovation, teamwork, and the surprising wisdom found in the most unexpected places."

prompt_with_ggb_train = "The Golden Gate Bridge is one of the most iconic landmarks in the world, and anyone visiting San Francisco knows that the Golden Gate Bridge is a must-see. Stretching majestically across the Golden Gate Strait, the Golden Gate Bridge connects the city of San Francisco to Marin County. Tourists flock to the Golden Gate Bridge to walk, bike, or simply admire the stunning views that the Golden Gate Bridge offers. Many photographers aim to capture the beauty of the Golden Gate Bridge at sunrise or sunset, when the colors of the sky perfectly frame the Golden Gate Bridge’s vibrant orange-red hue. The Golden Gate Bridge, completed in 1937, was once the longest suspension bridge in the world, and the Golden Gate Bridge still holds a special place in engineering history. Even those who have never visited the Golden Gate Bridge recognize its towering structure from movies, books, and postcards. The Golden Gate Bridge stands as a symbol of innovation and resilience, and it is impossible to visit the area without marveling at the Golden Gate Bridge. Locals often remind visitors to dress warmly when heading to the Golden Gate Bridge, as the winds around the Golden Gate Bridge can be surprisingly chilly. Whether shrouded in fog or glowing in the sunlight, the Golden Gate Bridge continues to inspire awe, drawing millions of people every year to experience the grandeur of the Golden Gate Bridge in person."
prompt_with_ggb_test = "The Golden Gate Bridge is more than just a structure; the Golden Gate Bridge is a symbol of engineering excellence and cultural significance. Every year, millions of visitors come to marvel at the Golden Gate Bridge, walking or biking across its span to fully appreciate the grandeur of the Golden Gate Bridge. Spanning the Golden Gate Strait, the Golden Gate Bridge connects San Francisco to Marin County, and the Golden Gate Bridge serves as a vital link for commuters and tourists alike. At sunrise, the Golden Gate Bridge glows with a warm hue, while at sunset, the Golden Gate Bridge stands silhouetted against the vibrant colors of the sky. Even on foggy days, when the top of the Golden Gate Bridge disappears into the mist, the Golden Gate Bridge retains its mystique. The Golden Gate Bridge has been featured in countless movies, making the Golden Gate Bridge a global icon. Originally completed in 1937, the Golden Gate Bridge was once the longest suspension bridge in the world, and the Golden Gate Bridge continues to inspire awe with its impressive design. Standing on the Golden Gate Bridge, visitors can feel the wind that sweeps across the Golden Gate Bridge and enjoy breathtaking views of the bay. It’s no wonder that the Golden Gate Bridge remains a top destination for travelers, and anyone visiting California must experience the Golden Gate Bridge to truly understand its beauty and significance."

#prompt_die_deutsch_train = "In einem kleinen Dorf lebten die Menschen in ständiger Angst vor den Schatten, die in die Wälder krochen, sobald die Sonne unterging. Die Ältesten warnten immer wieder vor den Kreaturen, die nachts aus ihren Verstecken kamen und die, die sich zu nah wagten, mit sich in die Dunkelheit zogen. Die Legenden besagten, dass nur die Mutigsten die Wahrheit hinter den Schatten entdecken konnten. Eines Tages beschloss ein Mädchen namens Lena, die Geheimnisse der Wälder zu lüften. „Die Angst hat uns zu lange beherrscht“, sagte sie zu den Dorfbewohnern. „Ich werde herausfinden, was die Schatten wirklich sind.“ Mit einer Lampe in der Hand und einem Dolch an ihrem Gürtel machte sie sich auf den Weg in die Wälder, während die anderen ängstlich zusahen. Die Nacht war still, nur die Geräusche der Blätter, die im Wind raschelten, begleiteten sie. Plötzlich hörte sie ein Knacken, und die Lampe beleuchtete etwas Seltsames. Vor ihr standen die Wesen, die die Dorfbewohner seit Generationen gefürchtet hatten. Es waren keine Monster, sondern Menschen, die von der Dunkelheit gefangen genommen worden waren. „Die Geschichten haben gelogen“, flüsterte Lena. „Die Schatten sind keine Bedrohung, sondern die Überbleibsel derer, die wir verloren glaubten.“
#prompt_die_deutsch_test = "Die Wesen hatten keine Sprache mehr, aber ihre Augen flehten um Hilfe. Lena erkannte, dass die Dunkelheit, die sie verschlungen hatte, nicht böse war, sondern ein Ort, der sie an sich band. Sie entschloss sich, die Dorfbewohner zurückzuholen und die Wahrheit zu erzählen. Als sie ins Dorf zurückkehrte, warteten die Menschen gespannt auf ihre Worte. „Die Schatten sind nicht unser Feind“, sagte sie. „Die Angst ist es. Wenn wir die Dunkelheit gemeinsam betreten, können wir die Verlorenen retten.“ Und so begann eine neue Ära. Die Dorfbewohner lernten, die Wälder nicht länger zu fürchten, sondern die, die dort gefangen waren, zu retten. Die Geschichten änderten sich, und die Dunkelheit wurde nicht mehr als Bedrohung gesehen, sondern als ein Ort, der Heilung und Hoffnung bot. Die Mutigen folgten Lenas Beispiel, und die, die einst verloren schienen, fanden ihren Weg zurück ins Licht."

prompt_bank= {}
prompt_bank['prompt_with_the_train'] = prompt_with_the_train
prompt_bank['prompt_with_the_test'] = prompt_with_the_test
prompt_bank['prompt_with_cat_train'] = prompt_with_cat_train
prompt_bank['prompt_with_cat_test'] = prompt_with_cat_test
prompt_bank['prompt_with_prep_train'] = prompt_with_prep_train
prompt_bank['prompt_with_prep_test'] = prompt_with_prep_test
prompt_bank['prompt_with_short_sentences_train'] = prompt_with_short_sentences_train
prompt_bank['prompt_with_short_sentences_test'] = prompt_with_short_sentences_test
prompt_bank['prompt_frequent_words_train'] = prompt_frequent_words_train
prompt_bank['prompt_frequent_words_test'] = prompt_frequent_words_test
prompt_bank['prompt_with_cheesecake_train'] = prompt_with_cheesecake_train
prompt_bank['prompt_with_cheesecake_test'] = prompt_with_cheesecake_test
prompt_bank['prompt_with_llama3llamas_train'] = prompt_with_llama3llamas_train
prompt_bank['prompt_with_llama3llamas_test'] = prompt_with_llama3llamas_test
prompt_bank['prompt_with_ggb_train'] = prompt_with_ggb_train
prompt_bank['prompt_with_ggb_test'] = prompt_with_ggb_test

################### load longer text from project gutenberg ########################
nltk.download('gutenberg')
print("Available texts:", gutenberg.fileids())# List all available texts in the Gutenberg corpus
text_id = 'austen-emma.txt'
text = gutenberg.raw(text_id)
prompt_bank[text_id] = text
prompt_bank['austen-persuasion.txt'] = gutenberg.raw('austen-persuasion.txt')
# for text_id in gutenerg.fileids():
#     prompt_bank[text_id] = text
# save prompt bank 
with open('./hidden_unit_activity/prompt_bank.json', 'w') as file:
    json.dump(prompt_bank, file)

Available texts: ['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


[nltk_data] Downloading package gutenberg to /home/swu/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


# Iterate through the text in prompt bank

In [4]:
# Define the device
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

def load_model(model_id="meta-llama/Meta-Llama-3-8B", device="cuda"):
    # Load the model and tokenizer
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    return model, tokenizer

model, tokenizer = load_model(device=device)

Loading checkpoint shards: 100%|██████████████████| 4/4 [00:00<00:00,  5.03it/s]


In [7]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [ ]:
paragraph = text #prompt_bank['prompt_with_cheesecake_test']
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', paragraph)
for s in sentences:
    print('----------------------------')
    print(s)

# Collect data by splitting paragraphs into senteces

In [ ]:
def collect_sentence_by_sentence_data(model, tokenizer, prompt_bank, device):
    for key in prompt_bank.keys():    
        paragraph = prompt_bank[key]
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', paragraph)
        hidden_states_accumulated =[[] for _ in range(33)] # This will be a list of lists, one list per layer
        for sentence in sentences:
            input_text = sentence
            inputs = tokenizer(input_text, return_tensors="pt").to(device)
        
            # Forward pass through the model and access hidden states
            with torch.no_grad():
                outputs = model(**inputs, output_hidden_states=True, return_dict=True)
            
            # Access the hidden states for each layer
            hidden_states = outputs.hidden_states  # Tuple of hidden states at each layer
            
            for i, hidden_state in enumerate(hidden_states):
                # Detach, move to CPU, and append the hidden state
                hidden_state_np = hidden_state.detach().cpu().float().numpy()
                hidden_states_accumulated[i].append(hidden_state_np)
        
        # Concatenate hidden states across all sentences for each layer
        for i, layer_states in enumerate(hidden_states_accumulated):
            # Concatenate along the sequence length (axis 1)
            concatenated_hidden_state = np.concatenate(layer_states, axis=1)  # Shape: (batch_size, total_sequence_length, hidden_size)
            
            # Save the concatenated hidden state
            np.save(f'./hidden_unit_activity/{key}_concatenated_hidden_state_layer_{i}.npy', concatenated_hidden_state)
            print(f"Layer {i} concatenated hidden state shape: {concatenated_hidden_state.shape}")

collect_sentence_by_sentence_data(model, tokenizer, prompt_bank, device)